In [1]:
import gym
import ale_py as a
import time
import numpy as np
import matplotlib.pyplot as plt
from AtariNet import *
from Atari_genetic import game_params,load_gen

In [2]:
#[print(i,j) for i,j in enumerate(gym.envs.registry.all())]

In [3]:
game = "Tetris"
cur_gen = 2
params = game_params(game)
numactions = params[0]
inshape = params[1]
mut_power = params[2]
test_size = params[3]
poolsizes = params[4]
numconvlayers = params[5]
trunc = params[6]

population,mutations,avg_rewards = load_gen(game,cur_gen)

In [4]:
ale = a.ALEInterface()
env = gym.make(f'ALE/{game}-v5')
env = gym.wrappers.GrayScaleObservation(env)

A.L.E: Arcade Learning Environment (version +a7a216c)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version +a7a216c)
[Powered by Stella]


In [6]:
seed = population[0]
mut = mutations[0]
torch.manual_seed(seed)
atarinet = AtariNetCONV(inshape=inshape,
                        poolsizes = poolsizes,
                        numconvlayers = numconvlayers, 
                        outsize = numactions)

atarinet.mutate(mut_power,mut)

In [55]:
for i_episode in range(1):
    obs = env.reset()
    prev_obs = np.zeros(obs.shape)
    time.sleep(0.5)
    for t in range(10000):
        env.render()

#         time.sleep(0.01)
        
        #Preprocessing
        observation = ProcessIm_DemonAttack(obs,prev_obs)    

        #Forward pass
        probs = atarinet.forward(observation)

        #Sample action space from probability distribution
        action = np.random.choice(numactions, 1, p = probs.detach().numpy())[0]
#         action = env.action_space.sample()
#         actions = [i for i in range(numactions)]
#         action = actions[i_episode]

        #Take action
        prev_obs = obs
        obs, reward, done, info = env.step(action)
        if reward <0:
            print(reward)
        
        
#         plt.imshow(MaxPool(observation-new_observation),cmap = "gray")
#         plt.show()
#         print(reward,info['lives'], observation.shape)
        if done:
            print(t)
            break
# print(env.action_space)
# print(env.observation_space)

267


In [8]:
np.mean(avg_rewards)

0.01705882352941176